In [1]:
x_train_file_path = 'train_x.csv/train_x.csv'
x_train_clean = 'train_x.csv/short.csv'
x_test_file_path = 'test_x.csv/test_x.csv'
y_train_file_path = 'train_y.csv'
sample_sub = 'sample_submission.csv'
cx_text = 'CleanedXtest.csv'

In [2]:
from keras.models import Sequential
from keras.layers import Dense
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.metrics import accuracy_score
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from keras.layers import Dense, SimpleRNN
from keras.layers import LSTM
from sklearn.preprocessing import StandardScaler
from keras.layers import Flatten, Dropout, BatchNormalization
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

In [3]:
# get the corrisponding labels
y_train = pd.read_csv(y_train_file_path, low_memory=False)
x_train = pd.read_csv(x_train_file_path, low_memory=False)
x_test = pd.read_csv(x_test_file_path, low_memory=False)
c_x_test = pd.read_csv(cx_text, low_memory=False)
x_t_c = pd.read_csv(x_train_clean, low_memory=False)
sample_set = pd.read_csv(sample_sub)

In [4]:
x_train = x_train.dropna(axis=1, how='all')
x_train = x_train.drop('cellattributevalue', axis=1)
x_train = x_train.drop('celllabel', axis=1)
x_train = x_train.drop('nursingchartcelltypevalname', axis=1)
x_train = x_train.drop('nursingchartvalue', axis=1)
x_train = x_train.drop('labmeasurenamesystem', axis=1)
x_train = x_train.drop('labname', axis=1)
x_train = x_train.drop('labresult', axis=1)
#x_train = x_train.dropna()

In [5]:
x_t_c = x_t_c.drop('cellattributevalue', axis=1)
x_t_c = x_t_c.drop('celllabel', axis=1)
x_t_c = x_t_c.drop('nursingchartcelltypevalname', axis=1)
x_t_c = x_t_c.drop('nursingchartvalue', axis=1)
x_t_c = x_t_c.drop('labmeasurenamesystem', axis=1)
x_t_c = x_t_c.drop('labname', axis=1)
x_t_c = x_t_c.drop('labresult', axis=1)
x_t_c = x_t_c.drop('offset', axis=1)
x_t_c = x_t_c.drop('Unnamed: 0', axis=1)

In [6]:
# endoced dummies
df4 = pd.get_dummies(x_t_c['ethnicity'])
x_t_c = pd.concat([x_t_c, df4], axis=1).reindex(x_t_c.index)
x_t_c.drop('ethnicity', axis=1, inplace=True)

df3 = pd.get_dummies(x_t_c['gender'])
x_t_c = pd.concat([x_t_c, df3], axis=1).reindex(x_t_c.index)
x_t_c.drop('gender', axis=1, inplace=True)

In [7]:
x_t_c = x_t_c.drop('Native American', axis=1)

In [8]:
x_t_c = x_t_c.fillna(0.0)
x_t_c['age'] = pd.to_numeric(x_t_c['age'], errors='coerce').fillna(90.0).astype('float32')
# loop through each column in the DataFrame
for col in x_t_c.columns:
    # check if the column data type is float64
    if x_t_c[col].dtype == 'float64':
        # convert the column to float32
        x_t_c[col] = x_t_c[col].astype('float32')

In [9]:
y_train = y_train.drop('Unnamed: 0', axis=1)

In [10]:
y_train

,patientunitstayid,hospitaldischargestatus
0,141764,0
1,141765,0
2,143870,0
3,144815,0
4,145427,0
...,...,...
2011,3351297,0
2012,3351763,0
2013,3352231,0
2014,3352333,0


In [11]:
x_t_c

,admissionheight,admissionweight,age,patientunitstayid,unitvisitnumber,African American,Asian,Caucasian,Hispanic,Other/Unknown,Female,Male
0,157.500000,0.000000,87.0,141764,2,0,0,1,0,0,1,0
1,157.500000,46.500000,87.0,141765,1,0,0,1,0,0,1,0
2,167.000000,77.500000,76.0,143870,1,0,0,1,0,0,0,1
3,172.699997,60.299999,34.0,144815,1,0,0,1,0,0,1,0
4,177.800003,91.699997,61.0,145427,1,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2011,159.000000,77.300003,52.0,3351297,1,1,0,0,0,0,0,1
2012,165.100006,134.500000,62.0,3351763,1,0,0,1,0,0,1,0
2013,177.800003,127.000000,41.0,3352231,1,1,0,0,0,0,0,1
2014,177.800003,68.300003,72.0,3352333,1,0,0,1,0,0,0,1


In [12]:
x_t_c.iloc[11]

admissionheight         157.5
admissionweight          58.5
age                      90.0
patientunitstayid    149713.0
unitvisitnumber           1.0
African American          0.0
Asian                     0.0
Caucasian                 1.0
Hispanic                  0.0
Other/Unknown             0.0
Female                    1.0
Male                      0.0
Name: 11, dtype: float64

In [17]:
y_label = y_train['hospitaldischargestatus']

In [13]:
c_x_test = c_x_test.drop('cellattributevalue', axis=1)
c_x_test = c_x_test.drop('celllabel', axis=1)
c_x_test = c_x_test.drop('nursingchartcelltypevalname', axis=1)
c_x_test = c_x_test.drop('nursingchartvalue', axis=1)
c_x_test = c_x_test.drop('labmeasurenamesystem', axis=1)
c_x_test = c_x_test.drop('labname', axis=1)
c_x_test = c_x_test.drop('labresult', axis=1)
c_x_test = c_x_test.drop('Unnamed: 0', axis=1)
c_x_test = c_x_test.drop('offset', axis=1)

# endoced dummies
df4 = pd.get_dummies(c_x_test['ethnicity'])
c_x_test = pd.concat([c_x_test, df4], axis=1).reindex(c_x_test.index)
c_x_test.drop('ethnicity', axis=1, inplace=True)

df3 = pd.get_dummies(c_x_test['gender'])
c_x_test = pd.concat([c_x_test, df3], axis=1).reindex(c_x_test.index)
c_x_test.drop('gender', axis=1, inplace=True)

c_x_test = c_x_test.fillna(0)

c_x_test['age'] = pd.to_numeric(c_x_test['age'], errors='coerce').fillna(90).astype('float32')

# loop through each column in the DataFrame
for col in c_x_test.columns:
    # check if the column data type is float64
    if c_x_test[col].dtype == 'float64':
        # convert the column to float32
        c_x_test[col] = c_x_test[col].astype('float32')

c_x_test

,admissionheight,admissionweight,age,patientunitstayid,unitvisitnumber,African American,Asian,Caucasian,Hispanic,Other/Unknown,Female,Male
0,149.899994,0.000000,59.0,151179,1,0,0,1,0,0,1,0
1,165.100006,86.800003,66.0,151900,1,0,0,1,0,0,1,0
2,170.199997,81.000000,41.0,152954,1,0,0,1,0,0,1,0
3,193.000000,163.800003,39.0,158056,1,0,0,1,0,0,0,1
4,162.600006,49.200001,20.0,159411,1,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
499,185.399994,140.600006,47.0,3343295,1,1,0,0,0,0,0,1
500,165.100006,129.800003,63.0,3344944,2,0,0,1,0,0,1,0
501,172.699997,75.000000,73.0,3347502,1,0,0,1,0,0,0,1
502,160.000000,49.439999,87.0,3348293,2,0,0,1,0,0,1,0


In [18]:
y_label

0       0
1       0
2       0
3       0
4       0
       ..
2011    0
2012    0
2013    0
2014    0
2015    0
Name: hospitaldischargestatus, Length: 2016, dtype: int64

In [26]:
# Create SVM classifier
lr = LinearRegression()
lr.fit(x_t_c, y_label)

# Use the model to make predictions on the test data
y_pred = lr.predict(c_x_test)

gb = GradientBoostingRegressor(n_estimators=100,max_features=12, learning_rate=0.1, max_depth=1, random_state=1)

gb.fit(x_t_c, y_label)

y_pred = gb.predict(c_x_test)

In [27]:
df_pred = pd.DataFrame(y_pred, columns=['hospitaldischargestatus'])

In [28]:
x_t_c.shape[1]

12

In [29]:
# Extract one column from the first dataframe
col = c_x_test['patientunitstayid']
#col
#df_pred
# Concatenate the column with the second dataframe
result = pd.concat([col, df_pred], axis=1)
result = result.dropna()
result

,patientunitstayid,hospitaldischargestatus
0,151179,0.041814
1,151900,0.018316
2,152954,0.008987
3,158056,-0.011582
4,159411,-0.004551
...,...,...
499,3343295,0.024591
500,3344944,0.039561
501,3347502,0.101261
502,3348293,0.126941


In [30]:
result.to_csv('result.csv', index=False)